<a href="https://colab.research.google.com/github/takiyama0617/technical_analysis/blob/master/EDINET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   XBRLファイルを取得する
    * XBRLファイルは、EDINETから取得する
    * EDINET APIを使用：https://disclosure.edinet-fsa.go.jp/EKW0EZ0015.html
1.   取得したXBRLファイルから必要な情報を取得する
1.   






In [0]:
import requests
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd


In [0]:

class Edinet:
    def __init__(self):
        self.base_url = "https://disclosure.edinet-fsa.go.jp"

    # 提出された書類を取得する
    def get_documents(self, _date, _type):
        count, retry = 0, 3
        url = self.base_url + "/api/v1/documents.json"
        params = {'date': _date, 'type': _type}
        while True:
            try:
                response = requests.get(url=url, params=params)
                return response.text
            except Exception:
                print(f"アクセスに失敗しました。[ {count} ]")
                if count < retry:
                    count += 1
                    time.sleep(2)
                    continue
                else:
                    raise

    def get_documents_between(self, _since, _until, _type):
        for _date in pd.date_range(_since, _until, freq="B", tz="Asia/Tokyo"):
            time.sleep(1)
            print(_date.strftime('%Y-%m-%d'))
            h = self.get_documents(_date.strftime('%Y-%m-%d'), _type)
            print(h)



In [40]:
edinet = Edinet()
h = edinet.get_documents("2020-04-01", 1)
print(h)

edinet.get_documents_between("2020-04-01", "2020-04-30", 1)

{
    "metadata":
        {
            "title": "提出された書類を把握するためのAPI",
            "parameter":
                {
                    "date": "2020-04-01",
                    "type": "1"
                },
            "resultset":
                {
                    "count": 255
                },
            "processDateTime": "2020-05-24 00:00",
            "status": "200",
            "message": "OK"
        }
}

2020-04-01
{
    "metadata":
        {
            "title": "提出された書類を把握するためのAPI",
            "parameter":
                {
                    "date": "2020-04-01",
                    "type": "1"
                },
            "resultset":
                {
                    "count": 255
                },
            "processDateTime": "2020-05-24 00:00",
            "status": "200",
            "message": "OK"
        }
}

2020-04-02
{
    "metadata":
        {
            "title": "提出された書類を把握するためのAPI",
            "parameter":
                {
                  